In [2]:
import pandas as pd
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

In [2]:
applied_data = pd.read_json("./train_data/Applied_data.json")
aptitude_data_1 = pd.read_json("./train_data/Aptitude.json")
aptitude_data_2 = pd.read_json("./train_data/Aptitude_2.json")
interview_1 = pd.read_json('./train_data/Interview.json')
interview_2 = pd.read_json('./train_data/Interview_Next_Stage_2.json')
oa_data = pd.read_json('./train_data/OA.json')
offer_data_1 = pd.read_json('./train_data/Offer.json')
offer_data_2 = pd.read_json('./train_data/Offer_2.json')
rejected_2 = pd.read_json('./train_data/Rejected_2.json')



applied_data['label'] = applied_data.apply(lambda x: 0,axis =1)
oa_data['label'] = oa_data.apply(lambda x: 1,axis =1)
aptitude_data_1['label'] = aptitude_data_1.apply(lambda x: 1,axis =1)
aptitude_data_2['label'] = aptitude_data_2.apply(lambda x: 1,axis =1)
interview_1['label'] = interview_1.apply(lambda x: 2,axis =1)
interview_2['label'] = interview_2.apply(lambda x: 2,axis =1)
offer_data_1['label'] = offer_data_1.apply(lambda x: 3,axis =1)
offer_data_2['label'] = offer_data_2.apply(lambda x: 3,axis =1)
# rejected['label'] = rejected.apply(lambda x: 4,axis =1)
rejected_2['label'] = rejected_2.apply(lambda x: 4,axis =1)

df_jobs = pd.concat([applied_data, aptitude_data_1, aptitude_data_2, interview_1, interview_2, oa_data, offer_data_1, offer_data_2,rejected_2])

def remove_n(row):
    text = row['body']
    text = text.replace('\n' ," ")
    return text
df_jobs['body'] = df_jobs.apply(lambda x: remove_n(x),axis=1)
final_df = df_jobs.sample(frac=1).reset_index(drop=True)

# def append_text(row):
#     subj = row['subject']
#     body = row['body']

#     text = "Subject: " + subj + " Body:" + body 
#     return text

# final_df['text'] = final_df.apply(lambda x: append_text(x),axis=1)
final_df.to_csv("./train_data/job_stage_data.csv",index=False)


In [3]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="./train_data/job_stage_data.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset['train'].to_pandas()

,label,subject,body
0,1,Next Steps: Sales Representative Position Asse...,"Hi Lily Thompson, Thank you for expressing in..."
1,1,Take the Analytical Thinking Test for Financia...,"Dear Daniel Wilson, We appreciate your intere..."
2,2,Next Steps: Final Interview for Data Analyst P...,"Hello Jessica Brown, Great news! You have suc..."
3,2,Final Interview Invitation for Human Resources...,"Dear Oliver Davis, Congratulations! We are pl..."
4,1,Next Steps: Cognitive Ability Test for Sales R...,"Hi Lily Thompson, Thank you for expressing in..."
...,...,...,...
102,1,Take the Financial Analyst Position Assessment...,"Dear Daniel Wilson, We appreciate your intere..."
103,0,Your Application Is in Good Hands,"Hi Rachel Miller, Thank you for taking the ti..."
104,3,Job Offer: Project Manager Position,"Hi Rachel Miller, Congratulations! We are del..."
105,2,Final Interview Invitation: Sales Representati...,"Hi Lily Thompson, Congratulations on making i..."


In [5]:
    # Load a SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset['train'],
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=4,
    num_iterations=30, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"body": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/30 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 6420
  Num epochs = 1
  Total optimization steps = 1605
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1605 [00:00<?, ?it/s]

In [6]:
trainer.model._save_pretrained("job_stage_classification_model")

In [3]:
# Download from Hub and run inference
model = SetFitModel.from_pretrained("job_stage_classification_model")
# Run inference

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
model.push_to_hub("karanamak/job-stage-classifier")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/karanamak/job-stage-classifier/tree/main/'

In [8]:
def infer_one_sentence(sentence):
        predtext = [sentence]
        predicted_class = model(predtext)
        return str(predicted_class)

In [15]:
infer_one_sentence("""

Dear Akshit,

 

Thank you for your interest in SAP. We are excited that you are taking steps to build your career with us. Our culture is driven by our most important asset – our people – who collaborate and innovate to make the world run better.

To help us understand you better, and to help you identify areas where you will be able to excel, we would like you to complete a voluntary online assessment. If you would like to complete the assessment, please follow the link below to access the test. Please be aware that if you have already done the assessment while completing your online application, there is no need to repeat it and this link will not work for you.
 

https://integration-talentcentral.eu.shl.com/Integration/ce/9c4da5f3dfaa4eae86511243dfcb8e25/?rid=54197939 

If you have completed the assessment, our Talent Attraction Team will review this together with your application details. You will also receive a copy of the assessment report for your review. If you decide not to complete the assessment, the process will continue and our Talent Attraction team will get back to you.

We hope that this will help you to identify the type of role where you will be happy and perform to your best.

Best Regards,

SAP Talent Attraction Team

Technical Support
""")

'tensor([1], dtype=torch.int32)'

In [9]:
infer_one_sentence(""" Dear Akshit,

We have received your application for the position of Intern - Data Science and are thrilled that you’d like to join Micron!

We are currently evaluating your experience and qualifications. Should you be shortlisted for this position, a recruiter will be in contact with you.

To view your application status, log into your Candidate Home

In the meantime, we encourage you to regularly visit our career site . Our “Get Matched” feature will instantly match your resume with any relevant, open vacancies.

We would be delighted to answer your questions! If you are interested in learning more about Micron you may interact with our team members here . You may also view prior Q&A, by theme and topic.

Once again, thank you for your time and consideration.

Micron Talent Acquisition Team""")

'tensor([0], dtype=torch.int32)'

In [14]:
infer_one_sentence("You have been shortlisted for the interview.Pls let me know when youare free.")

'tensor([2], dtype=torch.int32)'

In [11]:
infer_one_sentence("Thank you for your interest in the [Job Title] position at [Company Name]. We appreciate your time and effort throughout the interview process.\n\nAfter careful consideration, we regret to inform you that we have selected another candidate for the position. The decision was based on several factors, including the specific skills and experience required for the role, which we found to be more aligned with the selected candidate.\n\nWhile your application demonstrated [mention some strengths], our team ultimately decided to move forward with a candidate who possessed stronger qualifications and experience directly related to the job requirements.\n\nWe understand that job searching can be a challenging process, and we encourage you to continue developing your skills and experience. We appreciate your interest in [Company Name], and your application will be kept on file for potential future openings that may be a better fit.\n\nThank you for your understanding, and we wish you the best of luck in your job search.")

'tensor([4], dtype=torch.int32)'

In [12]:
infer_one_sentence("please send your NRIC and passport to move on further.")

'tensor([0], dtype=torch.int32)'

In [13]:
infer_one_sentence("""thank you for returning my call earlier! J

As shared, we are happy to offer you an internship opportunity with IHiS for the project: NLP modelling & visualisation on clinical textual data

Kindly refer to the details below for the internship details.

Allowance:                  $1800 per month
Period:                        9 Jan 2023 – 26 May 2023 (credit bearing internship)
Office hours:               830am to 6pm (Monday to Thursday) and 830am to 530pm (Friday), 1h lunch break
Location                      1 Maritime Square #12-01 Harbourfront Centre Singapore 099253""")

'tensor([3], dtype=torch.int32)'